In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from py2neo import Graph
from py2neo.data import Node, Relationship
from neo4j import GraphDatabase, basic_auth
import time
from wordcloud import WordCloud
import re

### Data acquisition

Recupero dei comments e submissions di Reddit dalla 01/01/2021 alla data 01/02/2021 e riferiti al subreddit 'wallstreetbets'. Le date sono fissate all'interno degli script python, in questo modo si evita di prelevare dati al di fuori dell'intervallo di tempo delimitato da tali date

In [ ]:
!python3 wsb_reddit/submission_pushshift.py

In [ ]:
!python3 wsb_reddit/comment_pushshift.py
comment

Merge di tutti i files json recuperati nei passi precedenti in un unico file csv per le submissions e un csv per i comments

In [7]:
!python3 wsb_reddit/organize_submissions.py

Done! Saved to...	wsb_reddit/data/submission_df_raw.csv


In [8]:
!python3 wsb_reddit/organize_comments.py

Done! Saved to...	wsb_reddit/data/comment_df_raw.csv


<br/><br/>
Importazione dei dati in dataframes di Pandas. I files csv sono stati creati dagli scripts python eseguiti in precedenza

In [3]:
comm_file = os.path.join('wsb_reddit', 'data', 'comment_df_raw.csv')
sub_file = os.path.join('wsb_reddit', 'data', 'submission_df_raw.csv')

Caricamento submissions e comments da csv a dataframe Pandas

In [4]:
submissions_df = pd.read_csv(sub_file, index_col=0)
submissions_df.shape

(18453, 16)

In [5]:
comments_df = pd.read_csv(comm_file, index_col=0)
comments_df.shape

(17787, 18)

## Data preparation

Si verifica che per le submissions il campo self_text è vuoto (contiene solo uno spazio). Stessa cosa per il campo title per i comments

In [6]:
len(submissions_df[~(submissions_df['self_text'] == ' ')].index)

0

In [7]:
len(comments_df[~(comments_df['title'] == ' ')].index)

0

Per uniformare i campi, si rimuove self_text da submissions_df e title da comments_df.
Si rinomina self_text di comments_df in title

In [8]:
del submissions_df["self_text"]
del comments_df["title"]

In [9]:
comments_df.rename(columns={"self_text": "title"}, inplace=True)
comments_df

,total_awards_received,author,author_premium,created_utc,flair,is_video,num_comments,score,title,is_submission,is_op,subreddit_subscribers,upvote_ratio,no_follow,id,link_id,parent_id
0,0,VacationLover1,True,1610160906,NaN,0,0,983,"Get off me, I’m a Robinhood hundredaire and up...",0,0,NaN,0,False,gim73ad,t3_kti7at,t3_kti7at
1,2,Megakittysnuggler,False,1610165200,NaN,0,0,962,My man knows a bad investment when he see one.,0,0,NaN,0,False,gimf04v,t3_kti7at,t3_kti7at
2,5,InstagramStockTrader,False,1610176776,NaN,0,0,955,Why does it look like a retarded elephant on a...,0,0,NaN,0,False,gimw3hb,t3_ktm80t,t3_ktm80t
3,2,arpatel530,False,1610150619,NaN,0,0,859,True legend. Every single penny was used for s...,0,0,NaN,0,False,gilmxmj,t3_ktejpt,t3_ktejpt
4,0,diffcalculus,True,1610171865,NaN,0,0,430,The $150 is *after* the 800%,0,0,NaN,0,False,gimpsz9,t3_kti7at,t1_gimjwqm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17782,0,-EzW,False,1610683815,NaN,0,0,185,He went full retard. You never go full retard,0,0,NaN,0,False,gjb6127,t3_kxhxry,t1_gjac00w
17783,0,0ptimusPrim0,True,1610670517,NaN,0,0,179,#To anyone who is new here:\n\nBuy shares and ...,0,1,NaN,0,False,gjah6iu,t3_kxhq3z,t3_kxhq3z
17784,0,ptgauth,True,1610683154,NaN,0,0,177,Dont have rights to the music sorry :/,0,1,NaN,0,False,gjb4vm3,t3_kxgpa3,t1_gjb1wvb
17785,0,Blitzkringe69,False,1610681508,NaN,0,0,176,spotify upload when?,0,0,NaN,0,False,gjb1wvb,t3_kxgpa3,t1_gjaghws


In [10]:
submissions_df

,total_awards_received,author,author_premium,created_utc,flair,is_video,num_comments,score,title,is_submission,is_op,subreddit_subscribers,id,upvote_ratio,no_follow
0,0,anonbutler,False,1610584736,News,0,84,40,r/WSB to Citi: Do You Feel In Charge?,1,1,1822882,kwu9hu,1.00,False
1,2,Jeffamazon,True,1610584967,Meme,1,377,35,$GME - End Game 🚀🚀🚀🚀🚀,1,1,1822935,kwuc3a,1.00,False
2,0,God_Hates_You_Too,False,1610584549,Gain,0,49,30,"I listened to you degenerates, chunked in $350...",1,1,1822829,kwu7gn,1.00,False
3,0,TaxationAttorney,False,1610586079,Discussion,0,110,28,Tendies and Taxes: Info for your wife's boyfriend,1,1,1823197,kwuo7q,0.97,False
4,0,anactualalien,True,1610582693,Meme,0,5,28,We fuckin won,1,1,1822361,kwtmqp,1.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18448,0,Frostayyyyy,False,1611005673,News,0,18,1,BREAKING: SkyBridge Capital and CEO @Scaramucc...,1,1,1888153,l04ejn,1.00,True
18449,0,AbjectRaise,False,1611014190,Discussion,0,4,1,PSA: Turn Off Stock Lending On Your Brokerage ...,1,1,1889358,l077vo,1.00,True
18450,0,VurseRBLX,False,1611008798,DD,0,12,1,Roblox Direct Listing in February,1,1,1888600,l05gpu,1.00,True
18451,0,NotoriousSR,False,1611009446,Discussion,0,17,1,GME Short Squeeze: My Exit Strategy,1,1,1888704,l05ono,1.00,True


<br/>
Verifica presenza valori mancanti nei campi id e title

In [11]:
len(submissions_df[submissions_df['id'].isna() | submissions_df['title'].isna()].index)

0

In [12]:
len(comments_df[comments_df['id'].isna() | comments_df['title'].isna()].index)

0

Le date sono in long, con il numero espresso in secondi a partire dal 01/01/1070.
<br/>
Inserimento di un campo con la data in formato human readable, trasformando il valore contenuto nel campo created_at_utc.
<br/><br/>
Data quality<br/>
L'operazione è utile anche per verificare la correttezza del valore inserito in tale campo. Vengono poi contollati valori min e max per verificare che siano all'interno del range temporale richiesto

In [13]:
submissions_df['created_at_utc'] = pd.to_datetime(submissions_df['created_utc'], unit='s', utc=True)
submissions_df['created_at_utc'] = submissions_df['created_at_utc'].astype(str)

In [14]:
comments_df['created_at_utc'] = pd.to_datetime(comments_df['created_utc'], unit='s', utc=True)
comments_df['created_at_utc'] = comments_df['created_at_utc'].astype(str)

In [18]:
min(submissions_df['created_at_utc'])

'2020-12-01 00:00:14+00:00'

In [19]:
min(comments_df['created_at_utc'])


'2020-12-01 00:00:10+00:00'

In [20]:
max(submissions_df['created_at_utc'])

'2021-01-31 23:53:15+00:00'

In [21]:
max(submissions_df['created_at_utc'])

'2021-01-31 23:53:15+00:00'

<br/><br/>
A campione viene selezionata una submission alla quale fa riferimento un comment tramite il parent_id (caratteri dopo l'underscore)

In [74]:
submissions_df[submissions_df['id'] == 'ktm80t']

,total_awards_received,author,author_premium,created_utc,flair,is_video,num_comments,score,title,is_submission,is_op,subreddit_subscribers,id,upvote_ratio,no_follow,created_at_utc
12494,0,TopShelfBlunts,False,1610176205,News,0,279,2,Took GM 50 years to come up with this? So bad ...,1,1,1790162,ktm80t,1.0,False,2021-01-09 07:10:05+00:00


<br/><br/>
Viene constatato che per i comments non viene valorizzato il campo 'subreddit subscribers'

In [75]:
comments_df[~comments_df['subreddit_subscribers'].isna()]

,total_awards_received,author,author_premium,created_utc,flair,is_video,num_comments,score,title,is_submission,is_op,subreddit_subscribers,upvote_ratio,no_follow,id,link_id,parent_id,created_at_utc


<br>
Verifica duplicati

In [76]:
len(submissions_df[submissions_df.duplicated(subset=['id'], keep=False)].index)

0

In [77]:
len(comments_df[comments_df.duplicated(subset=['id'], keep=False)].index)

0

Verifica e sostituzione caratteri 'a capo' e 'tabulazione' con uno spazio

In [78]:
len(submissions_df[submissions_df['title'].str.contains('\t|\n|\r', regex=True)].index)

0

In [79]:
len(comments_df[comments_df['title'].str.contains('\t|\n|\r', regex=True)].index)

3085

In [80]:
comments_df.replace('\n|\r|\t',' ', regex=True, inplace=True)

<br/><br/>
Per tutti gli altri campi con valori Na viene inserita una stringa vuota

In [81]:
submissions_df.isna().sum().sum()

512

In [82]:
comments_df.isna().sum().sum()

35574

In [83]:
submissions_df.fillna('', inplace=True)

In [84]:
comments_df.fillna('', inplace=True)

<br/><br/>
Vengono estratti gli authors dalle submissions e dai comments e poi uniti in un unico dataframe

In [85]:
sub_authors = pd.DataFrame(np.unique(submissions_df['author']),columns=['author'])
sub_authors

,author
0,--X0X0--
1,-84
2,-8500-
3,-AMZN
4,-AbellaDanger
...,...
13585,zxc123zxc123
13586,zxcvictorjs
13587,zyqzy
13588,zyztem


In [86]:
comm_authors = pd.DataFrame(np.unique(comments_df['author']),columns=['author'])
comm_authors

,author
0,--Kurwa--
1,--X0X0--
2,--orb
3,--redacted--
4,-84
...,...
10599,zulari
10600,zulufux999
10601,zurako91
10602,zwifter11


Unione dei due dataframes contenenti gli authors, eliminando nello stesso tempo i duplicati

In [87]:
authors = pd.concat([sub_authors,comm_authors]).drop_duplicates().reset_index(drop=True)
authors

,author
0,--X0X0--
1,-84
2,-8500-
3,-AMZN
4,-AbellaDanger
...,...
22410,zsn100
22411,ztw2002
22412,zulari
22413,zurako91


<br/><br/>
Salvataggio dati 'ripuliti' su file

In [88]:
submissions_df.to_csv('../data/reddit_wsb_submissions.csv')
comments_df.to_csv('../data/reddit_wsb_comments.csv')

In [89]:
authors.to_csv('../data/reddit_wsb_authors.csv')

## Data storage

Inserimento dati in neo4j

In [108]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


conn = Neo4jConnection(uri="bolt://54.90.165.10:7687", 
                       user="neo4j",              
                       pwd="clicks-haul-qualifier")

In [109]:
def insert_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.
    
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):

        res = conn.query(query, 
                         parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result

Create constraint

In [72]:
conn.query('CREATE CONSTRAINT authors IF NOT EXISTS ON (a:Author) ASSERT a.name IS UNIQUE')

[]

In [73]:
conn.query('CREATE CONSTRAINT submissions IF NOT EXISTS ON (a:Submission) ASSERT a.id IS UNIQUE')

[]

In [74]:
conn.query('CREATE CONSTRAINT comments IF NOT EXISTS ON (a:Comment) ASSERT a.id IS UNIQUE')

[]

<br/>
Inserimento authors eseguendo una query per ogni record: ci si aspetta un inserimento più lento di uno in modalità batch

In [ ]:
for index, row in authors.iterrows():
    conn.query('''
      CREATE (n:Author { 
        name: $name})
    ''', parameters = {'name': row['author']})

In [112]:
def add_comments(rows, batch_size=5000):
 
    query = '''
    UNWIND $rows as row
    MERGE (n:Comment { id: row.id})
    SET n.total_awards_received = row.total_awards_received,
        n.author = row.author,
        n.author_premium = row.author_premium,
        n.num_comments = row.num_comments,
        n.score = row.score,
        n.title = row.title,
        n.upvote_ratio = row.upvote_ratio,
        n.no_follow = row.no_follow,
        n.subreddit_subscribers = row.subreddit_subscribers,
        n.link_id = row.link_id,
        n.parent_id = row.parent_id,
        n.created_at_utc = row.created_at_utc

    RETURN count(distinct n) as total
    '''
    return insert_data(query, rows, batch_size)

Inserimento comments in modalità batch

In [ ]:
add_comments(comments_df)

In [115]:
def add_submissions(rows, batch_size=5000):
 
    query = '''
    UNWIND $rows as row
    MERGE (n:Submission { id: row.id })
    SET n.total_awards_received = row.total_awards_received,
        n.author = row.author,
        n.author_premium = row.author_premium,
        n.flair = row.flair,
        n.num_comments = row.num_comments,
        n.score = row.score,
        n.title = row.title,
        n.upvote_ratio = row.upvote_ratio,
        n.no_follow = row.no_follow,
        n.subreddit_subscribers = row.subreddit_subscribers,
        n.created_at_utc = row.created_at_utc
    
    RETURN count(distinct n) as total
    '''
    return insert_data(query, rows, batch_size)

Inserimento submissions in modalità batch

In [ ]:
add_submissions(submissions_df)

<br/>
Creazione indici

In [86]:
#There is a uniqueness constraint on (:Comment {id}), so an index is already created that matches this

#query = """
#CREATE INDEX FOR (c:Comment) ON (c.id)
#"""
#conn.query(query)

In [85]:
query = """
CREATE INDEX FOR (c:Comment) ON (c.parent_id)
"""
conn.query(query)

[]

In [89]:
#There is a uniqueness constraint on (:Submission {id}), so an index is already created that matches this.

#query = """
#CREATE INDEX FOR (s:Submission) ON (s.id)
#"""
#conn.query(query)

In [90]:
query = """
CREATE INDEX FOR (c:Comment) ON (c.author)
"""
conn.query(query)

[]

Creazione relazione 'COMMENTED' tra Author e Comment

In [91]:
queryAuthorComment = """
MATCH (a:Author),(c:Comment)
WHERE a.name = c.author
CREATE (a)-[:COMMENTED]->(c)
"""
conn.query(queryAuthorComment)

[]

Creazione relazione 'SUBMITTED' tra Author e Submission

In [92]:
queryAuthorSubmission = """
MATCH (a:Author),(c:Submission)
WHERE a.name = c.author
CREATE (a)-[:SUBMITTED]->(c)
"""
conn.query(queryAuthorSubmission)

(No data)

Creazione relazione 'CHILD_OF' tra Comment e Submission

In [93]:
queryCommentSubmission = """
MATCH (a:Comment),(c:Submission)
WHERE substring(a.parent_id, 3) = c.id
CREATE (a)-[:CHILD_OF]->(c)
"""
conn.query(queryCommentSubmission)

[]

Creazione relazione 'CHILD_OF' tra Comment e Comment

In [94]:
queryCommentComment = """
MATCH (a:Comment),(c:Comment)
WHERE substring(a.parent_id, 3) = c.id
CREATE (a)-[:CHILD_OF]->(c)
"""
conn.query(queryCommentComment)

[]

### Neo4j graph datascience library

Crezione graph per poi calcolare il valore di degree centrality

In [ ]:
query = """
CALL gds.graph.create(
  'authorCommentSubmission',    
  ['Author', 'Submission', 'Comment'],   
  ['SUBMITTED', 'COMMENTED', 'CHILD_OF'] 
)
YIELD
  graphName AS graph, nodeProjection, nodeCount AS nodes, relationshipCount AS rels
"""
conn.query(query)

In [82]:
query = '''
CALL gds.degree.stream('authorCommentSubmission')
  YIELD nodeId, score
  RETURN gds.util.asNode(nodeId).name AS name, score AS degree
ORDER BY degree DESC, name DESC
'''
conn.query(query)

[<Record name='[deleted]' degree=392.0>,
 <Record name='nosalute' degree=56.0>,
 <Record name='AutoModerator' degree=55.0>,
 <Record name='OverpricedBagel' degree=53.0>,
 <Record name='btoned' degree=40.0>,
 <Record name='zjz' degree=39.0>,
 <Record name='OptionsAndTren' degree=37.0>,
 <Record name='iTradeStalks' degree=32.0>,
 <Record name='PencesElectrician' degree=32.0>,
 <Record name='ava28' degree=31.0>,
 <Record name='FudgieThaWhale' degree=31.0>,
 <Record name='landmanpgh' degree=30.0>,
 <Record name='247drip' degree=28.0>,
 <Record name='ProbablyTrolling1' degree=27.0>,
 <Record name='HolderofFour' degree=26.0>,
 <Record name='_FUCK_THE_GIANTS_' degree=25.0>,
 <Record name='KhAiMeLioN' degree=25.0>,
 <Record name='HardtackOrange' degree=25.0>,
 <Record name='Clutch3131' degree=25.0>,
 <Record name='WSBVoteBot' degree=24.0>,
 <Record name='PlaneIntroduction8' degree=24.0>,
 <Record name='GoBeaversOSU' degree=23.0>,
 <Record name='wallthrowawaystreet' degree=22.0>,
 <Record name=

In [83]:
query = """
CALL gds.degree.write('authorCommentSubmission', { writeProperty: 'degree' })
"""
conn.query(query)

[<Record nodePropertiesWritten=58655 centralityDistribution={'p99': 6.000022888183594, 'min': 0.0, 'max': 392.00194549560547, 'mean': 0.7471325984321057, 'p90': 1.0, 'p50': 1.0, 'p999': 16.00011444091797, 'p95': 2.0000076293945312, 'p75': 1.0} writeMillis=515 postProcessingMillis=292 createMillis=0 computeMillis=0 configuration={'orientation': 'NATURAL', 'writeConcurrency': 4, 'writeProperty': 'degree', 'relationshipWeightProperty': None, 'nodeLabels': ['*'], 'sudo': False, 'relationshipTypes': ['*'], 'username': None, 'concurrency': 4}>]

### Text analysis
Viene eseguito il conteggio di tutte le parole nei comments e submissions per trovare quali siano le parole maggiormente ricorrenti e che abbiano un significato per l'analisi dell'andamenteo del titolo GME

In [96]:
texts = []
def remove_links(row):
    clean_tweet = re.sub("@[A-Za-z0-9_]+","", row['title'])
    clean_tweet = re.sub("#[A-Za-z0-9_]+","", clean_tweet)
    clean_tweet = re.sub(r"http\S+", "", clean_tweet)
    texts.append(clean_tweet)

In [97]:
submissions_df.apply(remove_links, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
18448    None
18449    None
18450    None
18451    None
18452    None
Length: 18453, dtype: object

In [98]:
allWords= ' '.join( [twts for twts in texts] )
words = WordCloud().process_text(allWords)
words_df = pd.DataFrame.from_dict(words, orient='index')

In [99]:
words_df.sort_values(by=[0], ascending=False, inplace=True)

In [100]:
pd.set_option('display.max_rows', 200)

In [101]:
display(words_df.head(100))

,0
GME,2058
moon,798
stock,798
PLTR,773
buy,704
WSB,668
s,613
BB,572
call,559
new,523


Comments

In [102]:
comments_df.apply(remove_links, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
17782    None
17783    None
17784    None
17785    None
17786    None
Length: 17787, dtype: object

In [103]:
allWords_comm= ' '.join( [twts for twts in texts] )
words_comm = WordCloud().process_text(allWords_comm)
words_df_comm = pd.DataFrame.from_dict(words_comm, orient='index')

In [104]:
words_df_comm.sort_values(by=[0], ascending=False, inplace=True)

In [105]:
display(words_df_comm.head(200))

,0
GME,2884
s,1959
PLTR,1678
stock,1613
will,1535
buy,1442
WSB,1286
fuck,1279
one,1258
now,1206
